## Code Snippets for Data Preprocessing



In [1]:
# Imports
import pandas as pd


In [68]:
# Read the raw data
dfRaw = pd.read_csv('/Users/daniellemckenney/Programming/erasmusCourses/ML/mlProjectsBirbs/mlProjectBirds/dataset.csv', sep="\t", on_bad_lines="warn",parse_dates=['eventDate'])

In [57]:
# Print some useful stats: 
print(dfRaw.info())
print(dfRaw.describe())
print(dfRaw.shape)
print(dfRaw.columns)
# Number of unique locations
print(dfRaw['locality'].value_counts())
# All entries should be within Spain
print(dfRaw['countryCode'].value_counts())
# No data is missing in the relevant columns:
print(df.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 753579 entries, 1980-09-11 to 2011-04-12
Data columns (total 49 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   gbifID                            753579 non-null  int64  
 1   datasetKey                        753579 non-null  object 
 2   occurrenceID                      753579 non-null  object 
 3   kingdom                           753579 non-null  object 
 4   phylum                            753579 non-null  object 
 5   class                             753579 non-null  object 
 6   order                             753579 non-null  object 
 7   family                            753579 non-null  object 
 8   genus                             753579 non-null  object 
 9   species                           753579 non-null  object 
 10  infraspecificEpithet              1000 non-null    object 
 11  taxonRank                         75

In [84]:
# Function Definitions
def cleanRawData(rawData, relevantColumns):
    # returns a dataframe with only the relevent columns and with invalid location entries removed
    df = rawData.filter(items=relevantColumns)
    df = df[(df['decimalLatitude'] != 0) & (df['decimalLongitude'] != 0)]
    return df

def fillInCounts0(data, dateCombination):
    # Fill in count=0 in the grouped summary
    # Get all unique combinations of the date combination
    existing_combinations = data[dateCombination].drop_duplicates()

    # Create a DataFrame with all unique species
    all_species_combinations = pd.DataFrame({'species': data['species'].unique()})

    # Cross join (cartesian product) to get all combinations of 'species', 'eventDate', and 'locality'
    all_combinations = pd.merge(existing_combinations.assign(key=1), all_species_combinations.assign(key=1), on='key').drop('key', axis=1)

    # Merge with the original DataFrame to get counts
    on = dateCombination.append('species')
    result_df = pd.merge(all_combinations, data, on=on, how='left').fillna(0)
    return result_df

In [88]:
relevantColumns = ['species','locality','decimalLatitude', 'decimalLongitude', 'eventDate']

df = cleanRawData(dfRaw, relevantColumns)

# Group by 'species', 'eventDate', and 'locality' and count the rows in each group
grouped_eventDate_df = df.groupby(['species', 'eventDate', 'locality']).size().reset_index(name='count')
# Fill in count=0 on days where a locality was visited but no entry was made for that species
grouped_eventDate_df = fillInCounts0(grouped_eventDate_df, ['eventDate', 'locality'])
print(grouped_eventDate_df[10:])

grouped_eventDate_df['weekOfYear'] = grouped_eventDate_df['eventDate'].dt.isocalendar().week
grouped_eventDate_df['month'] = grouped_eventDate_df['eventDate'].dt.month
grouped_eventDate_df['quarter'] = grouped_eventDate_df['eventDate'].dt.to_period('Q')
grouped_eventDate_df['year'] = grouped_eventDate_df['eventDate'].dt.year

grouped_weekOfYear_df = grouped_eventDate_df.groupby(['species', 'weekOfYear', 'year', 'locality']).size().reset_index(name='count')
grouped_weekOfYear_df = fillInCounts0(grouped_eventDate_df, ['weekOfYear', 'year', 'locality'])

grouped_month_df = grouped_eventDate_df.groupby(['species', 'month', 'year', 'locality']).size().reset_index(name='count')
grouped_weekOfYear_df = fillInCounts0(grouped_eventDate_df, ['month', 'year', 'locality'])

grouped_quarter_df = grouped_eventDate_df.groupby(['species', 'quarter', 'locality']).size().reset_index(name='count')
grouped_weekOfYear_df = fillInCounts0(grouped_eventDate_df, ['quarter', 'locality'])

grouped_year_df = grouped_eventDate_df.groupby(['species', 'year', 'locality']).size().reset_index(name='count')
grouped_weekOfYear_df = fillInCounts0(grouped_eventDate_df, ['year', 'locality'])

print(grouped_weekOfYear_df[10:])
print(grouped_month_df[10:])
print(grouped_quarter_df[10:])
print(grouped_year_df[10:])

        eventDate      locality                 species  count
10     1964-08-30       Milagro           Turdus merula    0.0
11     1964-08-30       Milagro       Turdus philomelos    0.0
12     1964-09-15  Alcantarilla         Merops apiaster    1.0
13     1964-09-15  Alcantarilla  Phylloscopus collybita    0.0
14     1964-09-15  Alcantarilla           Turdus merula    0.0
...           ...           ...                     ...    ...
592987 2019-08-13    Villamanta       Turdus philomelos    1.0
592988 2019-09-09      Anguiano         Merops apiaster    0.0
592989 2019-09-09      Anguiano  Phylloscopus collybita    0.0
592990 2019-09-09      Anguiano           Turdus merula    0.0
592991 2019-09-09      Anguiano       Turdus philomelos    1.0

[592982 rows x 4 columns]
        year               locality                 species  eventDate  count  \
10      1964                Milagro           Turdus merula 1964-08-30    0.0   
11      1964                Milagro       Turdus philom

In [87]:

chiffChaffWeek=  grouped_weekOfYear_df[(grouped_weekOfYear_df['species'] =='Phylloscopus collybita')]
print(chiffChaffWeek.info())
print(chiffChaffWeek[10:])



<class 'pandas.core.frame.DataFrame'>
Index: 148248 entries, 1 to 592989
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   year        148248 non-null  int32         
 1   locality    148248 non-null  object        
 2   species     148248 non-null  object        
 3   eventDate   148248 non-null  datetime64[ns]
 4   count       148248 non-null  float64       
 5   weekOfYear  148248 non-null  UInt32        
 6   month       148248 non-null  int32         
 7   quarter     148248 non-null  period[Q-DEC] 
dtypes: UInt32(1), datetime64[ns](1), float64(1), int32(2), object(2), period[Q-DEC](1)
memory usage: 8.6+ MB
None
        year                              locality                 species  \
35      1970                 P. NAT. LAGUNA GRANDE  Phylloscopus collybita   
53      1971                 P. NAT. LAGUNA GRANDE  Phylloscopus collybita   
54      1971                 P. NAT. LAGUNA GRANDE  Phyl

In [81]:
print(grouped_weekOfYear_df[10:])

                                                                        eventDate  \
species           weekOfYear year locality                                          
Merops apiaster   1          1960 A sainza                                      0   
                                  A xunqueira                                   0   
                                  A xunqueira de alba                           0   
                                  ACEBUCHAL-P. NAT. MASISMAS DEL ODIEL          0   
                                  AIGUABARREIG TER-BRUGENT                      0   
...                                                                           ...   
Turdus philomelos 53         2016 Zumarraga                                     0   
                                  Zumaya                                        0   
                                  Zurbano                                       0   
                                  Zuriain                        

In [38]:
# Checking locality vs coordinates
# filtered_df = df[df['locality'].isin(['Localidad confidencial'])]
# print(filtered_df.filter(items=['locality', 'decimalLatitude','decimalLongitude', 'species', 'eventDate']))
# It looks like all of the coordinates of the same locality match

                      locality  decimalLatitude  decimalLongitude  \
1026    Localidad confidencial              0.0               0.0   
2093    Localidad confidencial              0.0               0.0   
2094    Localidad confidencial              0.0               0.0   
3157    Localidad confidencial              0.0               0.0   
3158    Localidad confidencial              0.0               0.0   
...                        ...              ...               ...   
749907  Localidad confidencial              0.0               0.0   
750957  Localidad confidencial              0.0               0.0   
751984  Localidad confidencial              0.0               0.0   
751985  Localidad confidencial              0.0               0.0   
752987  Localidad confidencial              0.0               0.0   

                            species            eventDate  
1026    Marmaronetta angustirostris  2015-07-16T00:00:00  
2093    Marmaronetta angustirostris  1999-01-01T00:00